In [1]:
# いつもの
import tensorflow as tf
import numpy as np
import matplotlib as plt
from tensorflow.examples.tutorials.mnist import input_data

np.random.seed(20160704)
tf.set_random_seed(20160704)

/Users/yamamotomasaomi/.pyenv/versions/anaconda3-5.1.0/envs/makuwo/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: compiletime version 3.6 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.5
  return f(*args, **kwds)


Instructions for updating:
Use the retry module or similar alternatives.


In [2]:
mnist = input_data.read_data_sets("/tmp/data/", one_hot = True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [3]:
# 畳みこみフィルターの設定
num_filters1 = 32

# プレースホルダーの設定
x = tf.placeholder(tf.float32,[None, 784])
x_image = tf.reshape(x, [-1, 28, 28, 1])

W_conv1 = tf.Variable(tf.truncated_normal([5, 5, 1, num_filters1],
                                         stddev = 0.1))
h_conv1 = tf.nn.conv2d(x_image, W_conv1,
                      strides = [1, 1, 1, 1], padding = 'SAME')
b_conv1 = tf.Variable(tf.constant(0.1, shape=[num_filters1]))
h_conv1_cutoff = tf.nn.relu(h_conv1 + b_conv1)
h_pool1 = tf.nn.max_pool(h_conv1_cutoff, ksize=[1, 2 , 2 , 1],
                         strides = [1, 2, 2, 1], padding = 'SAME')

In [4]:
num_filters2 = 64

W_conv2 = tf.Variable(tf.truncated_normal([5, 5, num_filters1, num_filters2],
                                           stddev = 0.1))
h_conv2 = tf.nn.conv2d(h_pool1, W_conv2,
                       strides=[1, 1, 1, 1], padding='SAME')
b_conv2 = tf.Variable(tf.constant(0.1, shape=[num_filters2]))

h_conv2_cutoff = tf.nn.relu(h_conv2 + b_conv2)

h_pool2 = tf.nn.max_pool(h_conv2_cutoff, ksize=[1, 2, 2, 1],
                         strides= [1, 2, 2, 1], padding = 'SAME')

In [5]:
h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*num_filters2])

num_units1 = 7*7*num_filters2
num_units2 = 1024

w2 = tf.Variable(tf.truncated_normal([num_units1, num_units2]))
b2 = tf.Variable(tf.constant(0.1, shape=[num_units2]))
hidden2 = tf.nn.relu(tf.matmul(h_pool2_flat, w2) + b2)

keep_prob = tf.placeholder(tf.float32)
hidden2_drop = tf.nn.dropout(hidden2, keep_prob)

w0 = tf.Variable(tf.zeros([num_units2, 10]))
b0 = tf.Variable(tf.zeros([10]))
p = tf.nn.softmax(tf.matmul(hidden2_drop, w0) + b0)

In [6]:
t = tf.placeholder(tf.float32, [None, 10])
loss = -tf.reduce_sum(t * tf.log(p))
train_step = tf.train.AdadeltaOptimizer(0.0001).minimize(loss)
correct_prediction = tf.equal(tf.argmax(p, 1), tf.argmax(t, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

In [7]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
saver = tf.train.Saver()

In [8]:
i = 0
for _ in range (20000):
    i+=1
    batch_xs, batch_ts = mnist.train.next_batch(50) 
    sess.run(train_step,
             feed_dict={x:batch_xs, t: batch_ts, keep_prob:0.5})
    
    if i % 500 == 0:
        loss_vals, acc_vals = [], []
        for c in range(4):
            start = len(mnist.test.labels) // 4 * c
            end = len(mnist.test.labels) // 4 * (c+1)
            loss_val, acc_val = sess.run([loss,accuracy],feed_dict={x:mnist.test.images[start:end],t:mnist.test.labels[start:end],keep_prob:1.0})
            loss_vals.append(loss_val)
            acc_vals.append(acc_val)
        loss_val = np.sum(loss_vals)
        acc_val = np.mean(acc_vals)
        print('Step: %d, Loss:%f, Accuracy: %f' 
                % (i, loss_val, acc_val))
        saver.save(sess, '/Users/yamamotomasaomi/Documents/GitHub/my_mechanic_studies/cnn_session', global_step=i)

Step: 500, Loss:22894.863281, Accuracy: 0.270700
Step: 1000, Loss:22759.962891, Accuracy: 0.488500
Step: 1500, Loss:22610.958984, Accuracy: 0.550100
Step: 2000, Loss:22443.320312, Accuracy: 0.616400
Step: 2500, Loss:22256.781250, Accuracy: 0.656500
Step: 3000, Loss:22048.714844, Accuracy: 0.671500
Step: 3500, Loss:21816.433594, Accuracy: 0.699900
Step: 4000, Loss:21557.226562, Accuracy: 0.723300
Step: 4500, Loss:21272.753906, Accuracy: 0.736500
Step: 5000, Loss:20961.929688, Accuracy: 0.743300
Step: 5500, Loss:20621.003906, Accuracy: 0.759500
Step: 6000, Loss:20254.425781, Accuracy: 0.765400
Step: 6500, Loss:19864.082031, Accuracy: 0.773300
Step: 7000, Loss:19448.597656, Accuracy: 0.778700
Step: 7500, Loss:19014.181641, Accuracy: 0.783800
Step: 8000, Loss:18561.314453, Accuracy: 0.788000
Step: 8500, Loss:18097.074219, Accuracy: 0.792600
Step: 9000, Loss:17619.943359, Accuracy: 0.794800
Step: 9500, Loss:17140.115234, Accuracy: 0.795400
Step: 10000, Loss:16657.271484, Accuracy: 0.800900


In [9]:
!ls cnn_session*

cnn_session-18000.data-00000-of-00001 cnn_session-19000.meta
cnn_session-18000.index               cnn_session-19500.data-00000-of-00001
cnn_session-18000.meta                cnn_session-19500.index
cnn_session-18500.data-00000-of-00001 cnn_session-19500.meta
cnn_session-18500.index               cnn_session-20000.data-00000-of-00001
cnn_session-18500.meta                cnn_session-20000.index
cnn_session-19000.data-00000-of-00001 cnn_session-20000.meta
cnn_session-19000.index
